##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Chat with SQL using LangChain

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ranasaurus9/gemini-workshop/blob/main/4-Chat_with_SQL_using_langchain.ipynb"><img src = "../../images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

Reading an SQL database can be challenging for humans. However, with accurate prompts, Gemini models can generate answers based on the data. Through the use of the Gemini API, you will be able retrieve necessary information by chatting with a SQL database.

In [ ]:
!pip install -U -q google-generativeai langchain langchain-community langchain-google-genai

In [ ]:
import sqlite3

from langchain.chains import create_sql_query_chain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

import google.generativeai as genai
from IPython.display import Markdown

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.


In [ ]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## Setting up the database
To query a database, you first need to set one up.

1. **Load the California Housing Dataset:** Load the dataset from sklearn.datasets and extract it into a DataFrame.


In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing_bunch = fetch_california_housing(as_frame=True)
california_housing_df = california_housing_bunch.frame

2. **Connect to the SQLite database:** The database will be stored in the specified file.

In [ ]:
conn = sqlite3.connect("mydatabase.db")

# Write the DataFrame to a SQL table named 'housing'.
california_housing_df.to_sql("housing", conn, index=False)

In [ ]:
# Create an SQLDatabase object
db = SQLDatabase.from_uri("sqlite:///mydatabase.db")

## Question to query
With the database connection established, the `SQLDatabase` object now contains information about our database, which the model can access.

You can now start asking the LLM to generate queries.


In [ ]:
# you can see what information is available
Markdown(db.get_table_info())

In [ ]:
# Define query chain
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
write_query_chain = create_sql_query_chain(llm, db)

You use `create_sql_query_chain` that fits our database. It provides default prompts for various types of SQL including Oracle, Google SQL, MySQL and more.


In this case, default prompt is suitable for the task. However, feel free to experiment with writing this part of our chain yourself to suit your preferences.

In [ ]:
Markdown(write_query_chain.get_prompts()[0].template)

In [ ]:
response = write_query_chain.invoke({"question": "What is the total population?"})
response

In [ ]:
db.run('SELECT SUM("Population") FROM housing')

Great! The SQL query is correct, but it needs proper formatting before it can be executed directly by the database.


## Validating the query
You will pass the output of the previous query to a model that will extract just the SQL query and ensure its validity.

In [ ]:
validate_prompt = PromptTemplate(
    input_variables=["not_formatted_query"],
    template="""You are going to receive a text that contains a SQL query. Extract that query.
    Make sure that it is a valid SQL command that can be passed directly to the Database.
    Avoid using Markdown for this task.
    Text: {not_formatted_query}"""
)

In [ ]:
validate_chain = write_query_chain | validate_prompt | llm | StrOutputParser()
validate_chain.invoke({"question": "What is the total population?"})

## Automatic querying
Now, let's automate the process of querying the database using *QuerySQLDataBaseTool*. This tool can receive text from previous parts of the chain, execute the query, and return the answer.


In [ ]:
execute_query = QuerySQLDataBaseTool(db=db)
execute_chain = validate_chain | execute_query
execute_chain.invoke({"question": "What is the total population?"})

## Generating answer
You are almost done!

To enhance our output, you'll use LLM not only to get the number but to get properly formatted and natural language response.

In [ ]:
answer_prompt = PromptTemplate.from_template(
    """You are going to receive a original user question, generated SQL query, and result of said query. You should use this information to answer the original question. Use only information provided to you.

Original Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer_chain = (
    RunnablePassthrough.assign(query=validate_chain).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt | llm | StrOutputParser()
)

answer_chain.invoke({"question": "What is the total population?"})

## Next steps

Congratulations! You've successfully created a functional chain to interact with SQL. Now, feel free to explore further by asking different questions.